In [ ]:
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from common import numerical_recipes as numrec
from common.baseclasses import ArrayWithAxes as AWA
from IPython.display import HTML

%matplotlib notebook

In [ ]:
class BesselGenerator():

    """
    Generator of Bessel functions placed at arbitrary "center" position
    within an xy mesh space.  Works by generating a bessel function on
    a much larger xy mesh and then translating/truncating to the
    original mesh.  Nothing fancy, but good for performance.
    """

    def __init__(self,func,q=20,\
                 xs=np.linspace(-1,1,101),\
                 ys=np.linspace(-1,1,101),\
                 N_tip_eigenbasis = 10,
                 radial=True,
                 debug = True):

        #Bookkeeping of the coordinate mesh
        self.func = func
        self.xs,self.ys=xs,ys
        self.shape=(len(xs),len(ys))
        self.midx=self.xs[self.shape[0]//2]
        self.midy=self.ys[self.shape[1]//2]
        self.dx=np.max(self.xs)-np.min(self.xs)
        self.dy=np.max(self.ys)-np.min(self.ys)
        if debug:
            print("midx: {}\nmidy: {}\ndx: {}\ndy: {}".format(self.midx, self.midy, self.dx, self.dy))
        
        #Make a mesh grid twice bigger in each direction
        bigshape=[2*N-1 for N in self.shape]
        xs2grid,ys2grid=np.ogrid[-self.dx:+self.dx:bigshape[0]*1j,
                                 -self.dy:+self.dy:bigshape[1]*1j]
        self.xs2=xs2grid.squeeze()
        self.ys2=ys2grid.squeeze()

        rs2 = np.sqrt(xs2grid**2+ys2grid**2)
        if debug:
            print("rs2: {}".format(rs2))
        if radial:
            self.bigJ = self.func(q*rs2)
        else:
            self.bigJ = self.func(xs2grid,ys2grid)

    def __call__(self,x0,y0):

        #@ASM2019.12.21 There were previously some bugs here
        #The center of our function is presently (midx,midy)
        shift_by_dx=x0-self.midx
        shift_by_dy=y0-self.midy
        shift_by_nx=int(self.shape[0]*shift_by_dx/self.dx)
        shift_by_ny=int(self.shape[1]*shift_by_dy/self.dy)

        newJ=np.roll(np.roll(self.bigJ,shift_by_nx,axis=1),\
                     shift_by_ny,axis=0)
        output = newJ[self.shape[0]//2:(3*self.shape[0])//2,\
                     self.shape[1]//2:(3*self.shape[1])//2]
        return AWA(output,axes=[self.xs,self.ys])


In [ ]:
def Bessel(n):
    return lambda r: sp.jv(n,r)
def PlaneWave(kx,ky):
    return lambda x,y: np.sin(kx*x+ky*y)
gen = BesselGenerator(Bessel(0), debug=False)
#gen = BesselGenerator(PlaneWave(1,10), radial=False, debug=False)
#plt.figure()
#plt.imshow(gen(0,0), extent = [-1,1,-1,1])

In [ ]:
x0,y0 = -1,-1
curr = gen(x0,y0)
extent=[curr.axes[0].min(),curr.axes[1].max(),curr.axes[1].min(),curr.axes[1].max()]
fig = plt.figure()
im = plt.imshow(curr, extent=extent)

# initialization function: plot the background of each frame
def init():
    im.set_data(curr)
    return [im]

# animation function.  This is called sequentially
def animate(i):
    x0 =  (i/50)%2-1
    y0 = (((i/50)//2)/10-1)%2-1
    im.set_array(gen(x0,y0))
    return [im]

#anim = animation.FuncAnimation(fig, animate, init_func=init,frames=10000, interval=1, blit=True)

#HTML(anim.to_html5_video())

